# Setup

- The anchor atoms are at the top of `./wat6/bgbg-tip3p/a-bam-p/a00/a-bam-p.pdb` (H1 H2 H3 G1 G2)
- (From `SETUP-Restraints.pl`) The static restraint distance force constant is 5.0
- The static restratint angle force constant is 100.0
- The static restraint ? force constant is 6.0
- Angles 104.3/108.8

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

import logging
from importlib import reload
reload(logging)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p')

import glob as glob
import os as os
import subprocess as sp
import shutil as shutil
from itertools import compress

import parmed as pmd
import paprika
from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint
from paprika.restraints import amber_restraint_line
from paprika.restraints import create_window_list
from paprika.utils import make_window_dirs

from smirnovert.convert import convert
from smirnovert.utils import create_pdb_with_conect
from smirnovert.utils import prune_conect
from smirnovert.utils import extract_water_and_ions
from smirnovert.utils import create_water_and_ions_parameters
from smirnovert.utils import load_mol2
from smirnovert.utils import split_topology
from smirnovert.utils import create_host_guest_topology

from openforcefield.typing.engines.smirnoff import ForceField, unit
from openforcefield.utils import mergeStructure

logging.info("Started logging...")
logging.info("pAPRika version: " + paprika.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using `git` to find current version`.


2018-08-02 12:09:17 PM Started logging...
2018-08-02 12:09:17 PM pAPRika version: --0.0.3


In [1]:
from constants import systems

AttributeError: 'list' object has no attribute 'split'

In [33]:
def rsync_pdb(system):

    source = os.path.join(
        "/data/nhenriksen/projects/cds/wat6/bgbg-tip3p", system, "a00"
    )
    destination = os.path.join("systems", system, "bgbg-tip3p")

    if not os.path.isdir(destination):
        print(f"Creating {destination}...")
        os.makedirs(destination)

    log_file = os.path.join(destination, "rsync.log")
    rsync_list = [
        "rsync",
        "-armv",
        '-e "ssh"',
        '--include="{}"'.format(system + ".pdb"),
        '--exclude="*"',
        "{}".format("davids4@kirkwood:" + source + "/"),
        "{}".format("."),
    ]
    with open(log_file, "a") as file:
        file.write(" ".join(rsync_list))
        file.write("\n")
        p = sp.Popen(
            [" ".join(rsync_list)],
            cwd=destination,
            stdout=file,
            stderr=file,
            shell=True,
        )
        output, error = p.communicate()
        if p.returncode == 0:
            pass
        elif p.returncode == 1:
            print("Output: {}".format(output))
            print("Error: {}".format(error))


In [34]:
def grep_anchor_atoms(system):
    pdb = os.path.join("systems", system, "bgbg-tip3p", system + ".pdb")
    with open(pdb, "r") as file:
        remark = file.readline()
    remark = remark.rstrip()
    remark = remark.split(" ")

    prmtop = pmd.load_file(
        os.path.join("systems", system, "smirnoff", "smirnoff.prmtop")
    )
    dummy_residues = prmtop[":DUM"].residues
    host_residue = prmtop[":MGO"].residues[0].number + 1
    guest_residue = prmtop[f":{system.split('-')[1].upper()}"].residues[0].number + 1

    anchor_atoms = {
        "D1": f":{dummy_residues[0].number + 1}",
        "D2": f":{dummy_residues[1].number + 1}",
        "D3": f":{dummy_residues[2].number + 1}",
        "H1": f":{host_residue}@{remark[2].split('@')[1]}",
        "H2": f":{host_residue + 2}@{remark[3].split('@')[1]}",
        "H3": f":{host_residue + 4}@{remark[4].split('@')[1]}",
        "G1": f":{guest_residue}@{remark[5].split('@')[1]}",
        "G2": f":{guest_residue}@{remark[6].split('@')[1]}",
    }
    return anchor_atoms


In [35]:
def setup_static_restraints(
    anchor_atoms, windows, structure, distance_fc=5.0, angle_fc=100.0
):
    static_restraints = []
    static_restraint_atoms = [
        [anchor_atoms["D1"], anchor_atoms["H1"]],
        [anchor_atoms["D2"], anchor_atoms["D1"], anchor_atoms["H1"]],
        [anchor_atoms["D1"], anchor_atoms["H1"], anchor_atoms["H2"]],
        [
            anchor_atoms["D3"],
            anchor_atoms["D2"],
            anchor_atoms["D1"],
            anchor_atoms["H1"],
        ],
        [
            anchor_atoms["D2"],
            anchor_atoms["D1"],
            anchor_atoms["H1"],
            anchor_atoms["H2"],
        ],
        [
            anchor_atoms["D1"],
            anchor_atoms["H1"],
            anchor_atoms["H2"],
            anchor_atoms["H3"],
        ],
    ]

    for index, atoms in enumerate(static_restraint_atoms):
        this = static_DAT_restraint(
            restraint_mask_list=atoms,
            num_window_list=windows,
            ref_structure=structure,
            force_constant=angle_fc if len(atoms) > 2 else distance_fc,
            amber_index=True,
        )

        static_restraints.append(this)
    return static_restraints

In [36]:
def setup_guest_restraints(
    anchor_atoms,
    windows,
    structure,
    distance_fc=5.0,
    angle_fc=100.0,
    pull_initial=6.0,
    pull_final=24.0,
):
    guest_restraints = []

    guest_restraint_atoms = [
        [anchor_atoms["D1"], anchor_atoms["G1"]],
        [anchor_atoms["G2"], anchor_atoms["D1"], anchor_atoms["G1"]],
        [anchor_atoms["D1"], anchor_atoms["G1"], anchor_atoms["G2"]],
    ]
    guest_restraint_targets = {
        "initial": [pull_initial, 180.0, 180.0],
        "final": [pull_final, 180.0, 180.0],
    }

    for index, atoms in enumerate(guest_restraint_atoms):
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = structure
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach["fc_final"] = angle_fc
            this.release["fc_final"] = angle_fc
        else:
            this.attach["fc_final"] = distance_fc
            this.release["fc_final"] = angle_fc
        this.attach["target"] = guest_restraint_targets["initial"][index]
        this.attach["fraction_list"] = attach_fractions

        this.pull["target_final"] = guest_restraint_targets["final"][index]
        this.pull["num_windows"] = windows[1]
        
        this.release['target'] = guest_restraint_targets["final"][index]
        # Keep the guest restraints on during release.
        this.release['fraction_list'] = [1.0] * windows[2]

    
        this.initialize()

        guest_restraints.append(this)
    return guest_restraints

In [37]:
def setup_conformation_restraints(
    template, targets, windows, attach_fractions, structure, resname, fc=6.0
):

    conformational_restraints = []
    host_residues = len(structure[":{}".format(resname.upper())].residues)
    first_host_residue = structure[":{}".format(resname.upper())].residues[0].number + 1

    for n in range(first_host_residue, host_residues + first_host_residue):
        if n + 1 < host_residues + first_host_residue:
            next_residue = n + 1
        else:
            next_residue = first_host_residue

        for (index, atoms), target in zip(enumerate(template), targets):
            
            conformational_restraint_atoms = []
            if index == 0:
                conformational_restraint_atoms.append(f":{n}@{atoms[0]}")
                conformational_restraint_atoms.append(f":{n}@{atoms[1]}")
                conformational_restraint_atoms.append(f":{n}@{atoms[2]}")
                conformational_restraint_atoms.append(f":{next_residue}@{atoms[3]}")
            else:
                conformational_restraint_atoms.append(f":{n}@{atoms[0]}")
                conformational_restraint_atoms.append(f":{n}@{atoms[1]}")
                conformational_restraint_atoms.append(f":{next_residue}@{atoms[2]}")
                conformational_restraint_atoms.append(f":{next_residue}@{atoms[3]}")

            this = DAT_restraint()
            this.auto_apr = True
            this.amber_index = True
            this.topology = structure
            this.mask1 = conformational_restraint_atoms[0]
            this.mask2 = conformational_restraint_atoms[1]
            this.mask3 = conformational_restraint_atoms[2]
            this.mask4 = conformational_restraint_atoms[3]

            this.attach["fraction_list"] = attach_fractions
            this.attach["target"] = target
            this.attach["fc_final"] = fc
            this.pull["target_final"] = target
            this.pull["num_windows"] = windows[1]
            
            this.release["fraction_list"] = attach_fractions[::-1]
            this.release["target"] = target
            this.release["fc_final"] = fc

            this.initialize()
            conformational_restraints.append(this)
    return conformational_restraints

In [38]:
def setup_guest_wall_restraints(template, targets, structure, resname, angle_fc=500.0, distance_fc=50.0):

    guest_wall_restraints = []
    host_residues = len(structure[":{}".format(resname.upper())].residues)
    first_host_residue = structure[":{}".format(resname.upper())].residues[0].number + 1

    for n in range(first_host_residue, host_residues + first_host_residue):

        for (index, atoms), target in zip(enumerate(template), targets):
            guest_wall_restraint_atoms = []
            if index == 0 or index == 1:
                guest_wall_restraint_atoms.append(f":{n}@{atoms[0]}")
                guest_wall_restraint_atoms.append(f"{atoms[1]}")
                angle = False
            else:
                guest_wall_restraint_atoms.append(f"{atoms[0]}")
                guest_wall_restraint_atoms.append(f"{atoms[1]}")
                guest_wall_restraint_atoms.append(f"{atoms[2]}")
                angle = True
                
                
            this = DAT_restraint()
            this.auto_apr = True
            this.amber_index = True
            this.topology = structure
            this.mask1 = guest_wall_restraint_atoms[0]
            this.mask2 = guest_wall_restraint_atoms[1]
            if angle:
                this.mask3 = guest_wall_restraint_atoms[2]
                this.attach["fc_initial"] = angle_fc
                this.attach["fc_final"] = angle_fc
                this.custom_restraint_values["rk2"] = 500.0
                this.custom_restraint_values["rk3"] = 0.0
            else:
                this.attach["fc_initial"] = distance_fc
                this.attach["fc_final"] = distance_fc
                this.custom_restraint_values["rk2"] = 50.0
                this.custom_restraint_values["rk3"] = 50.0
                this.custom_restraint_values["r1"] = 0.0
                this.custom_restraint_values["r2"] = 0.0

            this.attach["target"] = target
            this.attach["num_windows"] = windows[1]

            this.initialize()
            guest_wall_restraints.append(this)
    return guest_wall_restraints

In [39]:
attach_string = "0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00"
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = "0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00"
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

release_fractions = attach_fractions[::-1]

windows = [len(attach_fractions), len(pull_distances), len(release_fractions)]
print(f"There are {windows} windows in this attach-pull-release calculation.")

There are [15, 46, 15] windows in this attach-pull-release calculation.


In [40]:
system = systems[0]

In [41]:
rsync_pdb(system)

In [42]:
structure = pmd.load_file(os.path.join("systems", system, "smirnoff", "smirnoff.prmtop"),
                   os.path.join("systems", system, "smirnoff", "smirnoff.inpcrd"),
                    structure=True)


Using the slow `prmtop` reader! -DRS


In [43]:
anchor_atoms = grep_anchor_atoms(system)

Using the slow `prmtop` reader! -DRS


In [44]:
static_restraints = setup_static_restraints(
    anchor_atoms, windows, structure, distance_fc=5.0, angle_fc=100.0
)


In [45]:
guest_restraints = setup_guest_restraints(
    anchor_atoms,
    windows,
    structure,
    distance_fc=5.0,
    angle_fc=100.0,
    pull_initial=6.0,
    pull_final=24.0,
)


In [46]:
host_conformational_template = [["O5", "C1", "O1", "C4"], ["C1", "O1", "C4", "C5"]]
host_conformational_targets = [104.30, -108.8]

In [47]:
conformational_restraints = setup_conformation_restraints(
    host_conformational_template,
    host_conformational_targets,
    windows,
    attach_fractions,
    structure,
    resname="MGO",
    fc=6.0,
)


In [48]:
guest_wall_template = [
    ["O2", anchor_atoms["G1"]],
    ["O6", anchor_atoms["G1"]],
    [anchor_atoms["D2"], anchor_atoms["G1"], anchor_atoms["G2"]],
]
guest_wall_targets = [11.3, 13.3, 80.0]


In [49]:
guest_wall_restraints = setup_guest_wall_restraints(
    guest_wall_template,
    guest_wall_targets,
    structure,
    resname="MGO",
    angle_fc=500.0,
    distance_fc=50.0,
)


In [56]:
restraints = static_restraints + conformational_restraints + guest_restraints + guest_wall_restraints
for restraint in restraints:
    print(amber_restraint_line(restraint, "attach", 13))


&rst iat= 144,10,           r1=    0.00000, r2=    7.49023, r3=    7.49023, r4=  999.00000, rk2=    5.00000, rk3=    5.00000,  &end

&rst iat= 145,144,10,       r1=    0.00000, r2=  131.36554, r3=  131.36554, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,10,43,        r1=    0.00000, r2=   55.11101, r3=   55.11101, r4=  180.00000, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 146,145,144,10,   r1=  -64.88460, r2=  115.11540, r3=  115.11540, r4=  295.11540, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 145,144,10,43,    r1=  -15.40894, r2=  164.59106, r3=  164.59106, r4=  344.59106, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 144,10,43,101,    r1=  -70.84633, r2=  109.15367, r3=  109.15367, r4=  289.15367, rk2=  100.00000, rk3=  100.00000,  &end

&rst iat= 16,1,3,33,        r1=  -75.70000, r2=  104.30000, r3=  104.30000, r4=  284.30000, rk2=    4.48800, rk3=    4.48800,  &end

&rst iat= 1,3,33,35,        r1= -288.80000, r2= -108.80000, r3= -108.

- Make the window list
- Write all the restraints
- Get the timings
- Write the simulation script

In [25]:
for system in systems:
    window_list = create_window_list(guest_restraints)
    for window in window_list:
        shutil.rmtree(os.path.join("systems", system, "smirnoff", window))
        if not os.path.isdir(os.path.join("systems", system, "smirnoff", window)):
            print(f'Creating {os.path.join("systems", system, "smirnoff", window)}')
            os.makedirs(os.path.join("systems", system, "smirnoff", window))

2018-07-12 09:39:37 AM Restraints appear to be consistent
2018-07-12 09:39:37 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent


Creating systems/a-bam-p/smirnoff/a000
Creating systems/a-bam-p/smirnoff/a001
Creating systems/a-bam-p/smirnoff/a002
Creating systems/a-bam-p/smirnoff/a003
Creating systems/a-bam-p/smirnoff/a004
Creating systems/a-bam-p/smirnoff/a005
Creating systems/a-bam-p/smirnoff/a006
Creating systems/a-bam-p/smirnoff/a007
Creating systems/a-bam-p/smirnoff/a008
Creating systems/a-bam-p/smirnoff/a009
Creating systems/a-bam-p/smirnoff/a010
Creating systems/a-bam-p/smirnoff/a011
Creating systems/a-bam-p/smirnoff/a012
Creating systems/a-bam-p/smirnoff/a013
Creating systems/a-bam-p/smirnoff/p000
Creating systems/a-bam-p/smirnoff/p001
Creating systems/a-bam-p/smirnoff/p002
Creating systems/a-bam-p/smirnoff/p003
Creating systems/a-bam-p/smirnoff/p004
Creating systems/a-bam-p/smirnoff/p005
Creating systems/a-bam-p/smirnoff/p006
Creating systems/a-bam-p/smirnoff/p007
Creating systems/a-bam-p/smirnoff/p008
Creating systems/a-bam-p/smirnoff/p009
Creating systems/a-bam-p/smirnoff/p010
Creating systems/a-bam-p/

2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent


Creating systems/a-chp-p/smirnoff/p014
Creating systems/a-chp-p/smirnoff/p015
Creating systems/a-chp-p/smirnoff/p016
Creating systems/a-chp-p/smirnoff/p017
Creating systems/a-chp-p/smirnoff/p018
Creating systems/a-chp-p/smirnoff/p019
Creating systems/a-chp-p/smirnoff/p020
Creating systems/a-chp-p/smirnoff/p021
Creating systems/a-chp-p/smirnoff/p022
Creating systems/a-chp-p/smirnoff/p023
Creating systems/a-chp-p/smirnoff/p024
Creating systems/a-chp-p/smirnoff/p025
Creating systems/a-chp-p/smirnoff/p026
Creating systems/a-chp-p/smirnoff/p027
Creating systems/a-chp-p/smirnoff/p028
Creating systems/a-chp-p/smirnoff/p029
Creating systems/a-chp-p/smirnoff/p030
Creating systems/a-chp-p/smirnoff/p031
Creating systems/a-chp-p/smirnoff/p032
Creating systems/a-chp-p/smirnoff/p033
Creating systems/a-chp-p/smirnoff/p034
Creating systems/a-chp-p/smirnoff/p035
Creating systems/a-chp-p/smirnoff/p036
Creating systems/a-chp-p/smirnoff/p037
Creating systems/a-chp-p/smirnoff/p038
Creating systems/a-chp-p/

2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent
2018-07-12 09:39:38 AM Restraints appear to be consistent


...

Creating systems/b-mo3-s/smirnoff/a000
Creating systems/b-mo3-s/smirnoff/a001
Creating systems/b-mo3-s/smirnoff/a002
Creating systems/b-mo3-s/smirnoff/a003
Creating systems/b-mo3-s/smirnoff/a004
Creating systems/b-mo3-s/smirnoff/a005
Creating systems/b-mo3-s/smirnoff/a006
Creating systems/b-mo3-s/smirnoff/a007
Creating systems/b-mo3-s/smirnoff/a008
Creating systems/b-mo3-s/smirnoff/a009
Creating systems/b-mo3-s/smirnoff/a010
Creating systems/b-mo3-s/smirnoff/a011
Creating systems/b-mo3-s/smirnoff/a012
Creating systems/b-mo3-s/smirnoff/a013
Creating systems/b-mo3-s/smirnoff/p000
Creating systems/b-mo3-s/smirnoff/p001
Creating systems/b-mo3-s/smirnoff/p002
Creating systems/b-mo3-s/smirnoff/p003
Creating systems/b-mo3-s/smirnoff/p004
Creating systems/b-mo3-s/smirnoff/p005
Creating systems/b-mo3-s/smirnoff/p006
Creating systems/b-mo3-s/smirnoff/p007
Creating systems/b-mo3-s/smirnoff/p008
Creating systems/b-mo3-s/smirnoff/p009
Creating systems/b-mo3-s/smirnoff/p010
Creating systems/b-mo3-s/

2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent


Creating systems/b-mp3-p/smirnoff/p017
Creating systems/b-mp3-p/smirnoff/p018
Creating systems/b-mp3-p/smirnoff/p019
Creating systems/b-mp3-p/smirnoff/p020
Creating systems/b-mp3-p/smirnoff/p021
Creating systems/b-mp3-p/smirnoff/p022
Creating systems/b-mp3-p/smirnoff/p023
Creating systems/b-mp3-p/smirnoff/p024
Creating systems/b-mp3-p/smirnoff/p025
Creating systems/b-mp3-p/smirnoff/p026
Creating systems/b-mp3-p/smirnoff/p027
Creating systems/b-mp3-p/smirnoff/p028
Creating systems/b-mp3-p/smirnoff/p029
Creating systems/b-mp3-p/smirnoff/p030
Creating systems/b-mp3-p/smirnoff/p031
Creating systems/b-mp3-p/smirnoff/p032
Creating systems/b-mp3-p/smirnoff/p033
Creating systems/b-mp3-p/smirnoff/p034
Creating systems/b-mp3-p/smirnoff/p035
Creating systems/b-mp3-p/smirnoff/p036
Creating systems/b-mp3-p/smirnoff/p037
Creating systems/b-mp3-p/smirnoff/p038
Creating systems/b-mp3-p/smirnoff/p039
Creating systems/b-mp3-p/smirnoff/p040
Creating systems/b-mp3-p/smirnoff/p041
Creating systems/b-mp3-p/

2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent
2018-07-12 09:39:40 AM Restraints appear to be consistent


Creating systems/b-pb4-p/smirnoff/p019
Creating systems/b-pb4-p/smirnoff/p020
Creating systems/b-pb4-p/smirnoff/p021
Creating systems/b-pb4-p/smirnoff/p022
Creating systems/b-pb4-p/smirnoff/p023
Creating systems/b-pb4-p/smirnoff/p024
Creating systems/b-pb4-p/smirnoff/p025
Creating systems/b-pb4-p/smirnoff/p026
Creating systems/b-pb4-p/smirnoff/p027
Creating systems/b-pb4-p/smirnoff/p028
Creating systems/b-pb4-p/smirnoff/p029
Creating systems/b-pb4-p/smirnoff/p030
Creating systems/b-pb4-p/smirnoff/p031
Creating systems/b-pb4-p/smirnoff/p032
Creating systems/b-pb4-p/smirnoff/p033
Creating systems/b-pb4-p/smirnoff/p034
Creating systems/b-pb4-p/smirnoff/p035
Creating systems/b-pb4-p/smirnoff/p036
Creating systems/b-pb4-p/smirnoff/p037
Creating systems/b-pb4-p/smirnoff/p038
Creating systems/b-pb4-p/smirnoff/p039
Creating systems/b-pb4-p/smirnoff/p040
Creating systems/b-pb4-p/smirnoff/p041
Creating systems/b-pb4-p/smirnoff/p042
Creating systems/b-pb4-p/smirnoff/p043
Creating systems/b-pb4-p/

Now we need to translate the guest in each window.

In [19]:
for system in systems:
    window_list = create_window_list(guest_restraints)

attach_windows = [i[0] == "a" for i in window_list]
pull_windows = [i[0] == "p" for i in window_list]
release_windows = [i[0] == "r" for i in window_list]


2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:56:57 PM Restraints appear to be consistent
2018-07-13 12:

I could make the files in each window, but since these are already solvated, I realized that would lead to some clashes between the guests and the water. It seems more prudent to use the already prepared files and not re-solvate.
```
for system in systems:
    for window in compress(window_list, attach_windows):
        shutil.copy(
            os.path.join("systems", system, "smirnoff", "smirnoff.inpcrd"),
            os.path.join("systems", system, "smirnoff", window, "smirnoff.inpcrd"),
        )
        shutil.copy(
            os.path.join("systems", system, "smirnoff", "smirnoff.prmtop"),
            os.path.join("systems", system, "smirnoff", window, "smirnoff.prmtop"),
        )
    for window in compress(window_list, pull_windows):
        structure = pmd.load_file(
            os.path.join("systems", system, "smirnoff", "smirnoff.inpcrd")
        )
        window_index = compress(window_list, pull_windows).index(window)
        target_difference = (
            guest_restraints[0].phase["pull"]["targets"][window_index]
            - guest_restraints[0].pull["target_initial"]
        )

        for atom in structure.atoms:
            if atom.residue.name == str(system.split("-")[1].upper()):
                atom.xz += target_difference
        structure.save(
            os.path.join("systems", system, "smirnoff", window, "smirnoff.inpcrd"),
            overwrite=True,
        )
    for window in compress(window_list, release_windows):
        try:
            os.symlink(
                os.path.join("../", "p045", file),
                os.path.join("systems", system, "smirnoff", window, file),
            )
        except:
            pass
```

In [40]:
def rsync_solvated_systems(system, window_list):
    for window in window_list:
     
        
        if not os.path.isdir(os.path.join("systems", system, "smirnoff", window)):
            os.makedirs(os.path.join("systems", system, "smirnoff", window))
    
        if window[0] == "p":
            niel_prefix = "u"
        else:
            niel_prefix = "a"
        source = os.path.join(
            "/data/nhenriksen/projects/cds/wat6/bgbg-tip3p", system, niel_prefix + window[2:4]
        )
        destination = os.path.join("systems", system, "smirnoff", window)

        log_file = os.path.join(destination, "rsync.log")
        rsync_list = [
            "rsync",
            "-armv",
            '-e "ssh"',
            '--include="full.hmr.topo"',
            '--include="full.crds"',
            '--exclude="*"',
            "{}".format("davids4@kirkwood:" + source + "/"),
            "{}".format("."),
        ]
        with open(log_file, "a") as file:
            file.write(" ".join(rsync_list))
            file.write("\n")
            p = sp.Popen(
                [" ".join(rsync_list)],
                cwd=destination,
                stdout=file,
                stderr=file,
                shell=True,
            )
            print(rsync_list)
            output, error = p.communicate()
            if p.returncode == 0:
                pass
            elif p.returncode == 1:
                print("Output: {}".format(output))
                print("Error: {}".format(error))

In [41]:
for system in systems:
    rsync_solvated_systems(system, [i for i in window_list if i[0] == "a" or i[0] == "p"])

['rsync', '-armv', '-e "ssh"', '--include="full.hmr.topo"', '--include="full.crds"', '--exclude="*"', 'davids4@kirkwood:/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00/', '.']
['rsync', '-armv', '-e "ssh"', '--include="full.hmr.topo"', '--include="full.crds"', '--exclude="*"', 'davids4@kirkwood:/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a01/', '.']
['rsync', '-armv', '-e "ssh"', '--include="full.hmr.topo"', '--include="full.crds"', '--exclude="*"', 'davids4@kirkwood:/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a02/', '.']
['rsync', '-armv', '-e "ssh"', '--include="full.hmr.topo"', '--include="full.crds"', '--exclude="*"', 'davids4@kirkwood:/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a03/', '.']
['rsync', '-armv', '-e "ssh"', '--include="full.hmr.topo"', '--include="full.crds"', '--exclude="*"', 'davids4@kirkwood:/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a04/', '.']
['rsync', '-armv', '-e "ssh"', '--include="full.hmr.topo"', '--include

In [44]:
for system in systems:
    for window in compress(window_list, release_windows):
        for file in ["full.crds", "full.hmr.topo"]:
            try:
                os.symlink(
                    os.path.join("../", "p045", file),
                    os.path.join("systems", system, "smirnoff", window, file),
                )
            except:
                print(f"Problem creating symlink in {system} {window} {file}")

In [30]:
for system in systems:
    _ = convert(
            source=os.path.abspath(os.path.join("systems", system, "smirnoff", "a000")),
            destination=os.path.abspath(os.path.join("systems", system, "smirnoff", "a000")),
            prefix="smirnoff",
            reference_prmtop="full.hmr.topo",
            reference_inpcrd="full.crds",
            host_resname="MGO",
            guest_resname=system[2:5],
            dummy=True,
            debug=True
            )

    for window in compress(window_list, attach_windows):
        for file in ["smirnoff.inpcrd", "smirnoff.prmtop"]:
            try:
                os.symlink(
                    os.path.join("../", "a000", file),
                    os.path.join("systems", system, "smirnoff", window, file),
                )
            except:
                print(f"Problem creating symlink in {system} {window} {file}")


2018-07-13 06:35:04 AM Creating /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/smirnoff/a000/smirnoff.pdb with CONECT records...
2018-07-13 06:35:04 AM Pruning water-water CONECT records...
2018-07-13 06:35:04 AM Splitting topology into components...
2018-07-13 06:35:06 AM Creating a combined topology for the host and guest molecules...
2018-07-13 06:35:06 AM Writing a `mol2` for the host molecule...
2018-07-13 06:35:06 AM Writing a `mol2` for the host molecule...
2018-07-13 06:35:06 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/smirnoff/a000/MGO.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:35:06 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-p/smirnoff/a000/bam.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:35:06 AM Extracting water and ions from /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulati

Check if the host-guest parameter file already exists...
Check if the host-guest coordinate file already exists...
Check if solvated parameter file already exists...
Check if solvated coordinate file already exists...
Problem creating symlink in a-bam-p a000 smirnoff.inpcrd
Problem creating symlink in a-bam-p a000 smirnoff.prmtop
Problem creating symlink in a-bam-p a001 smirnoff.inpcrd
Problem creating symlink in a-bam-p a001 smirnoff.prmtop
Problem creating symlink in a-bam-p a002 smirnoff.inpcrd
Problem creating symlink in a-bam-p a002 smirnoff.prmtop
Problem creating symlink in a-bam-p a003 smirnoff.inpcrd
Problem creating symlink in a-bam-p a003 smirnoff.prmtop
Problem creating symlink in a-bam-p a004 smirnoff.inpcrd
Problem creating symlink in a-bam-p a004 smirnoff.prmtop
Problem creating symlink in a-bam-p a005 smirnoff.inpcrd
Problem creating symlink in a-bam-p a005 smirnoff.prmtop
Problem creating symlink in a-bam-p a006 smirnoff.inpcrd
Problem creating symlink in a-bam-p a006 

2018-07-13 06:35:09 AM Creating /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-s/smirnoff/a000/smirnoff.pdb with CONECT records...
2018-07-13 06:35:09 AM Pruning water-water CONECT records...
2018-07-13 06:35:09 AM Splitting topology into components...
2018-07-13 06:35:11 AM Creating a combined topology for the host and guest molecules...
2018-07-13 06:35:11 AM Writing a `mol2` for the host molecule...
2018-07-13 06:35:11 AM Writing a `mol2` for the host molecule...
2018-07-13 06:35:11 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-s/smirnoff/a000/MGO.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:35:11 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-bam-s/smirnoff/a000/bam.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:35:11 AM Extracting water and ions from /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulati

Check if the host-guest parameter file already exists...
Check if the host-guest coordinate file already exists...
Check if solvated parameter file already exists...
Check if solvated coordinate file already exists...
Problem creating symlink in a-bam-s a000 smirnoff.inpcrd
Problem creating symlink in a-bam-s a000 smirnoff.prmtop
Problem creating symlink in a-bam-s a001 smirnoff.inpcrd
Problem creating symlink in a-bam-s a001 smirnoff.prmtop
Problem creating symlink in a-bam-s a002 smirnoff.inpcrd
Problem creating symlink in a-bam-s a002 smirnoff.prmtop
Problem creating symlink in a-bam-s a003 smirnoff.inpcrd
Problem creating symlink in a-bam-s a003 smirnoff.prmtop
Problem creating symlink in a-bam-s a004 smirnoff.inpcrd
Problem creating symlink in a-bam-s a004 smirnoff.prmtop
Problem creating symlink in a-bam-s a005 smirnoff.inpcrd
Problem creating symlink in a-bam-s a005 smirnoff.prmtop
Problem creating symlink in a-bam-s a006 smirnoff.inpcrd
Problem creating symlink in a-bam-s a006 

2018-07-13 06:35:15 AM Creating /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-but-p/smirnoff/a000/smirnoff.pdb with CONECT records...
2018-07-13 06:35:15 AM Pruning water-water CONECT records...
2018-07-13 06:35:15 AM Splitting topology into components...
2018-07-13 06:35:16 AM Creating a combined topology for the host and guest molecules...
2018-07-13 06:35:16 AM Writing a `mol2` for the host molecule...
2018-07-13 06:35:16 AM Writing a `mol2` for the host molecule...
2018-07-13 06:35:16 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-but-p/smirnoff/a000/MGO.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:35:16 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/a-but-p/smirnoff/a000/but.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:35:16 AM Extracting water and ions from /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulati

...

Problem creating symlink in b-pha-p a000 smirnoff.inpcrd
Problem creating symlink in b-pha-p a000 smirnoff.prmtop


2018-07-13 06:45:33 AM Creating /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/b-pnt-s/smirnoff/a000/smirnoff.pdb with CONECT records...
2018-07-13 06:45:33 AM Pruning water-water CONECT records...
2018-07-13 06:45:33 AM Splitting topology into components...
2018-07-13 06:45:34 AM Creating a combined topology for the host and guest molecules...
2018-07-13 06:45:34 AM Writing a `mol2` for the host molecule...
2018-07-13 06:45:34 AM Writing a `mol2` for the host molecule...
2018-07-13 06:45:35 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/b-pnt-s/smirnoff/a000/MGO.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:45:35 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/b-pnt-s/smirnoff/a000/pnt.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:45:35 AM Extracting water and ions from /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulati

Problem creating symlink in b-pnt-s a000 smirnoff.inpcrd
Problem creating symlink in b-pnt-s a000 smirnoff.prmtop


2018-07-13 06:45:41 AM Creating /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/b-pnt-p/smirnoff/a000/smirnoff.pdb with CONECT records...
2018-07-13 06:45:41 AM Pruning water-water CONECT records...
2018-07-13 06:45:41 AM Splitting topology into components...
2018-07-13 06:45:43 AM Creating a combined topology for the host and guest molecules...
2018-07-13 06:45:43 AM Writing a `mol2` for the host molecule...
2018-07-13 06:45:43 AM Writing a `mol2` for the host molecule...
2018-07-13 06:45:43 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/b-pnt-p/smirnoff/a000/MGO.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:45:44 AM Converting /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulations/systems/b-pnt-p/smirnoff/a000/pnt.mol2 to SYBYL atom types via Antechamber...
2018-07-13 06:45:44 AM Extracting water and ions from /home/dslochower/hgst-3tb-data/projects/smirnoff-host-guest-simulati

Problem creating symlink in b-pnt-p a000 smirnoff.inpcrd
Problem creating symlink in b-pnt-p a000 smirnoff.prmtop


In [38]:
for system in systems:
    for window in compress(window_list, pull_windows):

        source = os.path.join("systems", system, "smirnoff", window)
        destination = os.path.join("systems", system, "smirnoff", window)
        reference_inpcrd = "full.crds"
        reference_prmtop = "full.hmr.topo"
        prefix = "smirnoff"
        host_resname = "MGO"
        guest_resname = system[2:5]

        if os.path.isfile(os.path.join("systems", system, "smirnoff", window, "smirnoff.inpcrd")):
            continue

        create_pdb_with_conect(
            solvated_pdb=os.path.join(source, reference_inpcrd),
            amber_prmtop=os.path.join(source, reference_prmtop),
            output_pdb=os.path.join(destination, prefix) + ".pdb",
        )

        prune_conect(
            input_pdb=prefix + ".pdb", output_pdb=prefix + ".pruned.pdb", path=destination
        )

        extract_water_and_ions(
            amber_prmtop=reference_prmtop,
            amber_inpcrd=reference_inpcrd,
            host_residue=":" + host_resname.upper(),
            guest_residue=":" + guest_resname.upper(),
            dummy_atoms=True,
            output_pdb="water_ions.pdb",
            path=destination,
        )

        create_water_and_ions_parameters(
            input_pdb="water_ions.pdb",
            output_prmtop="water_ions.prmtop",
            output_inpcrd="water_ions.inpcrd",
            dummy_atoms=True,
            path=destination,
        )

        try:
            os.remove("systems", system, "smirnoff", window, f"{guest_resname.upper()}-sybyl.mol2")
        except:
            pass
        for file in ["MGO-sybyl.mol2", f"{guest_resname}-sybyl.mol2"]:
            try:
                os.symlink(
                    os.path.join("../", "a000", file),
                    os.path.join("systems", system, "smirnoff", window, file),
                )
            except:
                print(f"Problem creating symlink.")

        host = load_mol2(
            filename=os.path.join(destination, host_resname) + "-sybyl.mol2",
            name=host_resname,
            add_tripos=True,
        )

        guest = load_mol2(
            filename=os.path.join(destination, guest_resname) + "-sybyl.mol2",
            name=guest_resname,
            add_tripos=False,
        )

        molecules = [host, guest]

        components = split_topology(
            file_name=os.path.join(destination, prefix) + ".pruned.pdb"
        )
                
        hg_topology = create_host_guest_topology(
            components, host_resname=host_resname, guest_resname=guest_resname
        )
        
        #
        for bond in hg_topology.bonds:
            if bond.atom1.residue.name == "MGO" and bond.atom2.residue.name == "MO3":
                bond.delete()
                print("Bad bond!")
            elif bond.atom1.residue.name == "MO3" and bond.atom2.residue.name == "MGO":
                bond.delete()
                print("Bad bond!")
        #
        
        ff = ForceField("forcefield/smirnoff99Frosst.offxml")
        try:
            openmm = ff.createSystem(
            hg_topology.topology,
            molecules,
            nonbondedCutoff=1.1 * unit.nanometer,
            ewaldErrorTolerance=1e-4,
        )
        except:
            print(f"Problem with {system} {window}!")
            continue

        hg_structure = pmd.openmm.topsystem.load_topology(
            hg_topology.topology, openmm, hg_topology.positions
        )
        water_and_ions = pmd.amber.AmberParm(
            os.path.join(destination, "water_ions.prmtop"),
            xyz=os.path.join(destination, "water_ions.inpcrd"),
        )

        merged = mergeStructure(hg_structure, water_and_ions)
        reference = pmd.load_file(
            os.path.join(source, reference_prmtop),
            xyz=os.path.join(source, reference_inpcrd),
        )

        merged.box = reference.box
        try:
            merged.save(os.path.join(destination, "smirnoff.prmtop"))
        except:
            print("Check if solvated parameter file already exists...")
        try:
            merged.save(os.path.join(destination, "smirnoff.inpcrd"))
        except:
            print("Check if solvated coordinate file already exists...")
            clean_up(
                destination=destination,
                host_resname=host_resname,
                guest_resname=guest_resname,
            )


In [ ]:
for system in systems:

    structure = pmd.load_file(os.path.join("systems", system, "smirnoff", "smirnoff.prmtop"),
                   os.path.join("systems", system, "smirnoff", "smirnoff.inpcrd"),
                    structure=True)
    
    rsync_pdb(system)
    anchor_atoms = grep_anchor_atoms(system)
    
    static_restraints = setup_static_restraints(
        anchor_atoms, windows, structure, distance_fc=5.0, angle_fc=100.0
    )

    guest_restraints = setup_guest_restraints(
        anchor_atoms,
        windows,
        structure,
        distance_fc=5.0,
        angle_fc=100.0,
        pull_initial=6.0,
        pull_final=24.0,
    )

    host_conformational_template = [["O5", "C1", "O1", "C4"], ["C1", "O1", "C4", "C5"]]
    host_conformational_targets = [104.30, -108.8]

    conformational_restraints = setup_conformation_restraints(
        host_conformational_template,
        host_conformational_targets,
        windows,
        attach_fractions,
        structure,
        resname="MGO",
        fc=6.0,
    )

    guest_wall_template = [
        ["O2", anchor_atoms["G1"]],
        ["O6", anchor_atoms["G1"]],
        [anchor_atoms["D2"], anchor_atoms["G1"], anchor_atoms["G2"]],
    ]
    guest_wall_targets = [11.3, 13.3, 80.0]

    guest_wall_restraints = setup_guest_wall_restraints(
        guest_wall_template,
        guest_wall_targets,
        structure,
        resname="MGO",
        angle_fc=500.0,
        distance_fc=50.0,
    )

    restraints = (
        static_restraints
        + conformational_restraints
        + guest_restraints
        + guest_wall_restraints
    )
    
    window_list = create_window_list(guest_restraints)
    attach_windows = [i[0] == "a" for i in window_list]
    pull_windows = [i[0] == "p" for i in window_list]
    
    for window in compress(window_list, attach_windows):
        print(system, window)
        restraints_file = os.path.join("systems", system, "smirnoff", window, "disang.rest")
        if os.path.isfile(restraints_file):
            os.remove(restraints_file)

        with open(
            restraints_file, "w"
        ) as file:
            for restraint in restraints:
                string = amber_restraint_line(restraint, "attach", int(window[1:]))
                if string is not None:
                    file.write(string)
                    

    for window in compress(window_list, pull_windows):
        with open(
            restraints_file, "a"
        ) as file:
            for restraint in guest_restraints:
                string = amber_restraint_line(restraint, "pull", int(window[1:]))
                if string is not None:
                    file.write(string)


Using the slow `prmtop` reader! -DRS
Using the slow `prmtop` reader! -DRS


2018-07-17 07:32:43 PM Restraints appear to be consistent


a-bam-p a000
a-bam-p a001
a-bam-p a002
a-bam-p a003
a-bam-p a004
a-bam-p a005
a-bam-p a006
a-bam-p a007
a-bam-p a008
a-bam-p a009
a-bam-p a010
a-bam-p a011
a-bam-p a012
a-bam-p a013
Using the slow `prmtop` reader! -DRS
Using the slow `prmtop` reader! -DRS


2018-07-17 07:32:51 PM Restraints appear to be consistent


a-bam-s a000
a-bam-s a001
a-bam-s a002
a-bam-s a003
a-bam-s a004
a-bam-s a005
a-bam-s a006
a-bam-s a007
a-bam-s a008
a-bam-s a009
a-bam-s a010
a-bam-s a011
a-bam-s a012
a-bam-s a013
Using the slow `prmtop` reader! -DRS
Using the slow `prmtop` reader! -DRS


...

a-nmh-s a000
a-nmh-s a001
a-nmh-s a002
a-nmh-s a003
a-nmh-s a004
a-nmh-s a005
a-nmh-s a006
a-nmh-s a007
a-nmh-s a008
a-nmh-s a009
a-nmh-s a010
a-nmh-s a011
a-nmh-s a012
a-nmh-s a013
Using the slow `prmtop` reader! -DRS
Using the slow `prmtop` reader! -DRS


2018-07-17 07:37:27 PM Restraints appear to be consistent


a-oct-p a000
a-oct-p a001
a-oct-p a002
a-oct-p a003
a-oct-p a004
a-oct-p a005
a-oct-p a006
a-oct-p a007
a-oct-p a008
a-oct-p a009
a-oct-p a010
a-oct-p a011
a-oct-p a012
a-oct-p a013
Using the slow `prmtop` reader! -DRS
Using the slow `prmtop` reader! -DRS


2018-07-17 07:37:34 PM Restraints appear to be consistent


a-oam-p a000
a-oam-p a001
a-oam-p a002
a-oam-p a003
a-oam-p a004
a-oam-p a005
a-oam-p a006
a-oam-p a007
a-oam-p a008
a-oam-p a009
a-oam-p a010
a-oam-p a011
a-oam-p a012
a-oam-p a013
Using the slow `prmtop` reader! -DRS


In [18]:
tmp = structure["!:WAT & !:Na+ & !:Cl-"]

In [19]:
tmp.residues

ResidueList([
	<Residue MGO[0]>
	<Residue MGO[1]>
	<Residue MGO[2]>
	<Residue MGO[3]>
	<Residue MGO[4]>
	<Residue MGO[5]>
	<Residue MGO[6]>
	<Residue PNT[7]>
	<Residue DUM[8]>
	<Residue DUM[9]>
	<Residue DUM[10]>
])

In [25]:
for system in systems:
    for window in window_list[0:15+46-1]:
        print(f"{system} {window}")
        if os.path.isfile(os.path.join("systems", system, "smirnoff", window, "n_atoms.dat")):
            print(f"Skipping")
            continue
        structure = pmd.load_file(os.path.join("systems", system, "smirnoff", window, "smirnoff.prmtop"),
                       os.path.join("systems", system, "smirnoff", window, "smirnoff.inpcrd"),
                        structure=True)
        vac = structure["!:WAT & !:Na+ & !:Cl-"]
        try:
            vac.save(os.path.join("systems", system, "smirnoff", window, "vac.prmtop"))
        except:
            pass
        try:
            vac.save(os.path.join("systems", system, "smirnoff", window, "vac.pdb"))
        except:
            pass
        with open(os.path.join("systems", system, "smirnoff", window, "n_atoms.dat"), "w") as f:
            f.write(f"{len(vac.atoms)}")
        


a-bam-p a000
Using the slow `prmtop` reader! -DRS
a-bam-p a001
Using the slow `prmtop` reader! -DRS
a-bam-p a002
Using the slow `prmtop` reader! -DRS
a-bam-p a003
Using the slow `prmtop` reader! -DRS
a-bam-p a004
Using the slow `prmtop` reader! -DRS
a-bam-p a005
Using the slow `prmtop` reader! -DRS
a-bam-p a006
Using the slow `prmtop` reader! -DRS
a-bam-p a007
Using the slow `prmtop` reader! -DRS
a-bam-p a008
Using the slow `prmtop` reader! -DRS
a-bam-p a009
Using the slow `prmtop` reader! -DRS
a-bam-p a010
Using the slow `prmtop` reader! -DRS
a-bam-p a011
Using the slow `prmtop` reader! -DRS
a-bam-p a012
Using the slow `prmtop` reader! -DRS
a-bam-p a013
Using the slow `prmtop` reader! -DRS
a-bam-p p000
Using the slow `prmtop` reader! -DRS
a-bam-p p001
Using the slow `prmtop` reader! -DRS
a-bam-p p002
Using the slow `prmtop` reader! -DRS
a-bam-p p003
Using the slow `prmtop` reader! -DRS
a-bam-p p004
Using the slow `prmtop` reader! -DRS
a-bam-p p005
Using the slow `prmtop` reader! -DRS


KeyboardInterrupt: 

**Modify ParmEd to work around: https://github.com/ParmEd/ParmEd/issues/991**

In [23]:
def write_vacuum_file(system):
    for window in window_list[0:15+46-1]:
        print(f"{system} {window}")
        if os.path.isfile(os.path.join("systems", system, "smirnoff", window, "n_atoms.dat")):
            print(f"Skipping")
            continue
        structure = pmd.load_file(os.path.join("systems", system, "smirnoff", window, "smirnoff.prmtop"),
                       os.path.join("systems", system, "smirnoff", window, "smirnoff.inpcrd"),
                        structure=True)
        vac = structure["!:WAT & !:Na+ & !:Cl-"]
        try:
            vac.save(os.path.join("systems", system, "smirnoff", window, "vac.prmtop"))
        except:
            pass
        try:
            vac.save(os.path.join("systems", system, "smirnoff", window, "vac.pdb"))
        except:
            pass
        with open(os.path.join("systems", system, "smirnoff", window, "n_atoms.dat"), "w") as f:
            f.write(f"{len(vac.atoms)}")

In [26]:
for system in systems:
    write_vacuum_file(system)

a-bam-p a000
Skipping
a-bam-p a001
Skipping
a-bam-p a002
Skipping
a-bam-p a003
Skipping
a-bam-p a004
Skipping
a-bam-p a005
Skipping
a-bam-p a006
Skipping
a-bam-p a007
Skipping
a-bam-p a008
Skipping
a-bam-p a009
Skipping
a-bam-p a010
Skipping
a-bam-p a011
Skipping
a-bam-p a012
Skipping
a-bam-p a013
Skipping
a-bam-p p000
Skipping
a-bam-p p001
Skipping
a-bam-p p002
Skipping
a-bam-p p003
Skipping
a-bam-p p004
Skipping
a-bam-p p005
Skipping
a-bam-p p006
Skipping
a-bam-p p007
Skipping
a-bam-p p008
Skipping
a-bam-p p009
Skipping
a-bam-p p010
Skipping
a-bam-p p011
Skipping
a-bam-p p012
Skipping
a-bam-p p013
Skipping
a-bam-p p014
Skipping
a-bam-p p015
Skipping
a-bam-p p016
Skipping
a-bam-p p017
Skipping
a-bam-p p018
Skipping
a-bam-p p019
Skipping
a-bam-p p020
Skipping
a-bam-p p021
Skipping
a-bam-p p022
Skipping
a-bam-p p023
Skipping
a-bam-p p024
Skipping
a-bam-p p025
Skipping
a-bam-p p026
Skipping
a-bam-p p027
Skipping
a-bam-p p028
Skipping
a-bam-p p029
Skipping
a-bam-p p030
Skipping
a-bam-p p0